# 1) Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


# 2) Load Dataset

In [3]:
data = pd.read_csv("customer_churn.csv")
print(data.head())


  customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0      CUST1    Male              1     Yes         No       7          Yes   
1      CUST2  Female              0      No         No      35           No   
2      CUST3    Male              0      No         No      55          Yes   
3      CUST4    Male              0      No         No      14          Yes   
4      CUST5    Male              0      No        Yes      10           No   

  InternetService        Contract  MonthlyCharges  TotalCharges  Churn  
0     Fiber optic        Two year           36.78       7397.48      1  
1     Fiber optic        Two year           32.04       6589.62      0  
2     Fiber optic  Month-to-month           26.76       2897.11      1  
3             DSL        Two year           92.40       5280.98      1  
4     Fiber optic        Two year          112.88       4344.48      1  


In [4]:
print(data.tail())

    customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
995    CUST996    Male              0     Yes         No      15          Yes   
996    CUST997    Male              1     Yes         No      37          Yes   
997    CUST998  Female              0     Yes         No       1          Yes   
998    CUST999  Female              0      No         No      67           No   
999   CUST1000    Male              0      No         No      24           No   

    InternetService  Contract  MonthlyCharges  TotalCharges  Churn  
995     Fiber optic  Two year           33.03       4808.91      1  
996              No  One year           84.34        808.25      1  
997     Fiber optic  One year           39.72       2765.14      0  
998              No  One year           29.76        808.64      1  
999     Fiber optic  One year           36.68       7696.20      1  


# 3) Data Preprocessing

# - Drop customerID if present

In [5]:
if 'customerID' in data.columns:
    data.drop('customerID', axis=1, inplace=True)


# - Convert categorical columns

In [6]:
label_encoder = LabelEncoder()
for col in data.select_dtypes(include='object'):
    data[col] = label_encoder.fit_transform(data[col])

# - Split X and y

In [7]:
X = data.drop('Churn', axis=1)
y = data['Churn']


# - Feature Scaling

In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

# -  Train-Test Split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# - Convert to tensors

In [11]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1,1)
y_test  = torch.tensor(y_test.values, dtype=torch.float32).view(-1,1)

# 4) ANN Model

In [12]:
class ChurnANN(nn.Module):
    def __init__(self, input_features):
        super(ChurnANN, self).__init__()
        self.fc1 = nn.Linear(input_features, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

model = ChurnANN(X_train.shape[1])


#**5**) Loss & Optimizer


In [13]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# 6) Training

In [14]:
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

print("Training Complete")


Epoch [10/50], Loss: 0.6972
Epoch [20/50], Loss: 0.6945
Epoch [30/50], Loss: 0.6923
Epoch [40/50], Loss: 0.6905
Epoch [50/50], Loss: 0.6889
Training Complete


# 7) Prediction & Accuracy

In [15]:
with torch.no_grad():
    predictions = model(X_test)
    predicted = (predictions > 0.5).float()
    accuracy = (predicted == y_test).sum() / y_test.shape[0]

print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 47.50%
